# Assignment 2, User-user collaborative filtering

In [1]:
import pandas as pd
import numpy as np

## Read file

In [70]:
df_movie_row = pd.read_excel(open("UUCF Assignment Spreadsheet.xls", "rb"), sheet_name="movie-row")
df_movie_row.set_index('Unnamed: 0', inplace=True)
df_movie_row.index.name = "Movie"
df_movie_row.columns.name = "User"
df_movie_row.head()

User,1648,5136,918,2824,3867,860,3712,2968,3525,4323,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
Movie,,,,,,,,,,,,,,,,,,,,,
11: Star Wars: Episode IV - A New Hope (1977),NaN,4.5,5.0,4.5,4.0,4.0,NaN,5.0,4.0,5.0,...,4.0,NaN,4.5,4.0,3.5,NaN,NaN,NaN,NaN,NaN
12: Finding Nemo (2003),NaN,5.0,5.0,NaN,4.0,4.0,4.5,4.5,4.0,5.0,...,4.0,NaN,3.5,4.0,2.0,3.5,NaN,NaN,NaN,3.5
13: Forrest Gump (1994),NaN,5.0,4.5,5.0,4.5,4.5,NaN,5.0,4.5,5.0,...,4.0,5.0,3.5,4.5,4.5,4.0,3.5,4.5,3.5,3.5
14: American Beauty (1999),NaN,4.0,NaN,NaN,NaN,NaN,4.5,2.0,3.5,5.0,...,4.0,NaN,3.5,4.5,3.5,4.0,NaN,3.5,NaN,NaN
22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),4.0,5.0,3.0,4.5,4.0,2.5,NaN,5.0,3.0,4.0,...,3.0,1.5,4.0,4.0,2.5,3.5,NaN,5.0,NaN,3.5


## Compute the correlation between every user

In [20]:
df_corr = df_movie_row.corr(method="pearson")
np.fill_diagonal(df_corr.values, 0)
df_corr.head()

User,1648,5136,918,2824,3867,860,3712,2968,3525,4323,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
User,,,,,,,,,,,,,,,,,,,,,
1648,0.000000,0.402980,-0.142206,0.517620,0.300200,0.480537,-0.312412,0.383348,0.092775,0.098191,...,-0.191988,0.493008,0.360644,0.551089,0.002544,0.116653,-0.429183,0.394371,-0.304422,0.245048
5136,0.402980,0.000000,0.118979,0.057916,0.341734,0.241377,0.131398,0.206695,0.360056,0.033642,...,0.488607,0.328120,0.422236,0.226635,0.305803,0.037769,0.240728,0.411676,0.189234,0.390067
918,-0.142206,0.118979,0.000000,-0.317063,0.294558,0.468333,0.092037,-0.045854,0.367568,-0.035394,...,0.373226,0.470972,0.069956,-0.054762,0.133812,0.015169,-0.273096,0.082528,0.667168,0.119162
2824,0.517620,0.057916,-0.317063,0.000000,-0.060913,-0.008066,0.462910,0.214760,0.169907,0.119350,...,-0.201275,0.228341,0.238700,0.259660,0.247097,0.149247,-0.361466,0.474974,-0.262073,0.166999
3867,0.300200,0.341734,0.294558,-0.060913,0.000000,0.282497,0.400275,0.264249,0.125193,-0.333602,...,0.174085,0.297977,0.476683,0.293868,0.438992,-0.162818,-0.295966,0.054518,0.464110,0.379856


# Part 1 - Without normalization

## Find top 5 neighbours and predict ratings

In [71]:
def find_neighbours(user, top_n, df_correlation):
    topn_neighbours = df_correlation.loc[user].nlargest(top_n)
    user_correl_col = df_correlation.loc[user]
    user_correl_col[user_correl_col < topn_neighbours.iloc[top_n - 1]] = 0
    return user_correl_col

def predict_movies_without_norm(user, neighbours, df_movie_ratings):
    df_movie_ratings = df_movie_ratings.fillna(0)
    sum_neighbour_ratings = df_movie_ratings @ neighbours
    
    df_movie_ratings[df_movie_ratings > 0] = 1
    sum_weights = df_movie_ratings @ neighbours
    
    movie_predictions = sum_neighbour_ratings/sum_weights
    movie_predictions.sort_values(ascending=False, inplace=True)
    return movie_predictions

## User #3712

In [72]:
neighbours = find_neighbours(89, 5, df_corr)
movie_predictions = predict_movies_without_norm(89, neighbours, df_movie_row)

In [73]:
movie_predictions

Movie
238: The Godfather (1972)                                                                              4.894124
278: The Shawshank Redemption (1994)                                                                   4.882194
807: Seven (a.k.a. Se7en) (1995)                                                                       4.774093
275: Fargo (1996)                                                                                      4.770944
424: Schindler's List (1993)                                                                           4.729056
122: The Lord of the Rings: The Return of the King (2003)                                              4.695962
13: Forrest Gump (1994)                                                                                4.601470
38: Eternal Sunshine of the Spotless Mind (2004)                                                       4.551396
453: A Beautiful Mind (2001)                                                                      

## Part 2 - Normalization

In [78]:
def predict_movies_with_norm(user, neighbours, df_movie_ratings):
    movie_ratings = df_movie_ratings.copy()
    user_mean = movie_ratings.mean(axis=0)
    norm_movie_ratings = movie_ratings - user_mean
    sum_neighbour_ratings = norm_movie_ratings.fillna(0) @ neighbours
    
    movie_rates_exist = df_movie_ratings.copy().fillna(0)
    movie_rates_exist[movie_rates_exist > 0] = 1
    sum_weights = movie_rates_exist @ neighbours
    
    movie_predictions = (sum_neighbour_ratings/sum_weights) + df_movie_ratings[[user]].mean().values[0]
    movie_predictions.sort_values(ascending=False, inplace=True)
    return movie_predictions

In [80]:
neighbours = find_neighbours(89, 5, df_corr)
predict_movies_with_norm(89, neighbours, df_movie_row)

Movie
238: The Godfather (1972)                                                                              5.322015
278: The Shawshank Redemption (1994)                                                                   5.261424
275: Fargo (1996)                                                                                      5.241111
807: Seven (a.k.a. Se7en) (1995)                                                                       5.201984
424: Schindler's List (1993)                                                                           5.199223
122: The Lord of the Rings: The Return of the King (2003)                                              5.186846
105: Back to the Future (1985)                                                                         5.073048
550: Fight Club (1999)                                                                                 5.069939
13: Forrest Gump (1994)                                                                           